##### Step B.1 --> Importing the required things 

In [1]:
import numpy as np
import tensorflow as tf 
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
from tensorflow.keras.datasets import cifar10
import matplotlib.pyplot as plt 


##### Step B.2 --> Loading the CIFAR - 10  dataset 

In [4]:
(_, _), (x_test, _) = cifar10.load_data()

x_data = x_test[:1000] #using the 1000 images only 
x_data = x_data.astype("float32")
print(x_data.shape)        # (1000, 32, 32, 3)

c:\Users\Rohini\Desktop\2 Unsupervised Image Orgniser\venv\Lib\site-packages\keras\src\datasets\cifar.py:18: VisibleDeprecationWarning: dtype(): align should be passed as Python or NumPy boolean but got `align=0`. Did you mean to pass a tuple to create a subarray type? (Deprecated NumPy 2.4)
  d = cPickle.load(f, encoding="bytes")


(1000, 32, 32, 3)


##### Step B.3 --> Resize the images (Critical Step)

In [6]:
# basicallt this step is resizing the every image in the dataset to 224 x 224 size, regardless of what size it is 

x_resized = tf.image.resize(x_data, (224, 224))
print(x_resized.shape)

(1000, 224, 224, 3)


##### Step B.4 --> Load the MobileNetV2 (Feature Extractor)

In [7]:
model = MobileNetV2(
    weights = "imagenet",
    include_top = False,            # this remove the classifier 
    pooling = "avg"                 # This step gives us 12000 vectors, fixed length embedding 
)

model.trainable = False             # interference only 

C:\Users\Rohini\AppData\Local\Temp\ipykernel_17560\2725545370.py:1: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  model = MobileNetV2(


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


##### Step B.5 --> Preprocess & Extract the Features 

In [ ]:
x_processed = preprocess_input(x_resized)
features = model.predict(x_processed, batch_size = 32, verbose = 1)

print("Feature Shape :", features.shape)

# the output feature shape : is the DNA of the each image  
# the 1280-D is a vector 

32/32 ━━━━━━━━━━━━━━━━━━━━ 19s 580ms/step
Feature Shape : (1000, 1280)


##### Step B.6 --> Sanity Checks 

##### Check the variance

In [ ]:
np.std(features) 

# if this is == 0, then something is broken, 
# if not zero, then the embeddings are meaningful

np.float32(0.709743)

##### Compare the two random images 

In [15]:
from numpy.linalg import norm 
a, b = np.random.choice(1000, 2, replace = False)
dist = norm(features[a] - features[b])
print("Feature Distance : ", dist)


# different images--> large distance, similar images --> smaller distance 


Feature Distance :  33.902473


##### Save the Articrafts so that we never recompute the embeddngs, streamlit app stays fast & clean seperation of concerns

In [17]:
np.save("../app/image_data.npy", x_data)
np.save("../app/image_features.npy", features)